In [1]:
%matplotlib inline

# Principal Components in Textual Analysis

## Problem

We would like to understand mutliple bodies of texts numerically and find a way to represent them that emphasizes the relevant differences. The hope is that this reveals previously unrecognized connections between diverse texts. 

The first step is to represent each text as a vector of numbers. This vector will almost necessarily contain _many_ dimensions.

### Numeric Text Representation

Our goal is to produce a $m\times n$ matrix with $n$ variables for each of the $m$ texts. This matrix will have the form below, as an example.

|Text|$v_1$|$\ldots$|$v_j$|$\ldots$|$v_n$|
|--|--|--|--|--|
|Text 1|5|...|100|...|9|
|...|...|...|...|...|...|
|Text m|10|...|87|...|3|

Let us call this matrix $A$. We will return to it later.

### High Dimensionality
There are two ways of representing each text numerically that I investigated. In _either_ case, the numeric representation of a text can have many hundreds or thousands of variables.

| Option | Description | Advantages | Disadvantages |
|--------|------------|---------------|---------------|
|Word frequency|$v_j$ is the frequency of occurrences of a particular word|Non-parametric|Loses context|
|Dictionary tagging|$v_j$ is the frequency of occurrences of a particular _set_ of words, usually chosen to have business meaning|Provides more business meaning|Parametric, requires a "good" dictionary to exist|

### Dimensionality Reduction
We seek a way to reduce the many variables associated to a text to 2 or 3 significant variables that explain the key dimensions along which the text varies. 

If $A$ is our original matrix:

|Text|$v_1$|$\ldots$|$v_j$|$\ldots$|$v_n$|
|--|--|--|--|--|
|Text 1|5|...|100|...|9|
|...|...|...|...|...|...|
|Text m|10|...|87|...|3|

We seek a "narrower" version, perhaps with just 3 variables: 

|Text|$v'_1$|$v'_2$|$v'_3$|
|--|--|--|--|--|
|Text 1|5|34|100|
|...|...|...|...|
|Text m|10|47|87|


### Shakespeare

Our body of texts for this analysis will be the plays and sonnets of William Shakespeare.  His work is an ideal subject for this analysis. His plays vary widely from each other but return to the same themes, sometimes even with the same characters. At the same times, his audiences and readers recognized him as working in a few distinct genres. The First Folio, which was the first major printing to combine most of his works, grouped the plays into three genres: Comedies, Histories and Tragedies.
* Comedies: All's Well That Ends Well, As You Like It, The Comedy of Errors, Cymbeline, Love's Labour's Lost, Measure for Measure, The Merchant of Venice, The Merry Wives of Windsor, A Midsummer Night's Dream, Much Ado About Nothing, Pericles, The Taming of the Shrew, The Tempest, Twelfth Night, The Two Gentlemen of Verona, The Two Noble Kinsmen, The Winter's Tale
* Histories: 1 Henry IV, 2 Henry IV, Henry V, 1 Henry VI, 2 Henry VI, 3 Henry VI, Henry VIII, King John, Richard II, Richard III
* Tragedies: Antony and Cleopatra, Coriolanus, Cymbeline, Hamlet, Julius Caesar, King Lear, Macbeth, Othello, Pericles, Romeo and Juliet, Timon of Athens, Titus Andronicus, Troilus and Cressida
Our question will be to analyze what features of these plays make them "Histories", "Tragedies" or "Comedies". Perhaps we will even find a different and more revealing categorization.

## Math
A very simple, intuitive, and popular method of dimensionality reduction is Principle Component Analysis. 

### Principal Component Analysis (PCA)
PCA creates new combinations of the variables in a way that represent most of the variance in the data. In particular, it is helpful to reduce a set of many variables into the key variables or combinations of variables that is most responsible for variances among texts. 

### PCA Motivation
#### Subtract the mean of each variable ####
Let $A$ be the complete matrix of the samples and their features where we have standardized each vector of observations by subtracting its sample mean. Therefore, we assume that each column of the matrix $A$ has mean 0.
$$
A = \begin{bmatrix} 
\vdots & \vdots &  \vdots & \vdots  & \vdots \\
\mathbf{v_1} & \ldots & \mathbf{v_j} & \ldots &  \mathbf{v_n}  \\
\vdots & \vdots &  \vdots & \vdots &  \vdots 
\end{bmatrix}
$$ 

#### Find the covariance matrix of A by calculating $A^TA$ ####
Note that one way to view each vector $\mathbf{v_j}$ is as samples of a random variable $V_j$. Then the sample covariance of two variables $V_j$ and $V_k$ is: 
$$
cov(V_j,V_k) = \frac{1}{m-1} \sum_{i=1}^m (v_{j,i} - \bar{v_j})(v_{k,i} - \bar{v_k})
$$

Note that since we have subtracted by the means, then $\bar{v_j} = 0$.  Therefore, this is equivalent to the vector formulation: 
$$
cov(V_j,V_k) = \frac{1}{m-1} \mathbf{v_j}^T \mathbf{v_k}
$$

Finally, note that $A^TA$ is the matrix of covariances (after dividing by $m-1$)! The diagonal of this matrix is made up of variances of each $v_j$ and the off-diagonal entries are the "cross-covariances" between each pair of variables.

$$
\frac{1}{m-1}\mathbf{A}^T \mathbf{A} = \frac{1}{m-1}\begin{bmatrix}
\mathbf{v_1}^T \mathbf{v_1} & & \vdots &   & \mathbf{v_n}^T \mathbf{v_1} \\
\vdots & \ddots &  \vdots &  & \vdots  \\
\mathbf{v_1}^T \mathbf{v_j} &  & \mathbf{v_j}^T \mathbf{v_j} &   &  \mathbf{v_n}^T \mathbf{v_j}  \\
\vdots &  & \vdots  & \ddots & \vdots  \\
\mathbf{v_1}^T \mathbf{v_n} &  & \vdots  &  & \mathbf{v_n}^T \mathbf{v_n} 
\end{bmatrix}
$$

#### Can we eliminate correlations between variables? ####
Now that we know how to represent the covariances of our variables, we wish to find a transformation of the matrix $A$ into new variables that have no correlation with each other. In linear algebra terms, this means that the off-diagonal entries of the covariance matrix are reduced to zero.  This allows us to represent our samples in terms of $n$ variables with _no_ _cross-correlations_ _among_ _the_ _variables_!  In some sense, each of these new variables will explain a distinct driver of the variance in the data set and they aren't "polluted" by interaction with other variables.

Is such a transformation possible?

#### Calculate eigenvectors of $A^TA$ ####
The answer is yes! This is where eigenvalues and eigenvectors appear.

**Theorem** For any $m\times n$ matrix, $A$, there exists a matrix $P$ such that $(AP)^T(AP)$ is a diagonal matrix.

**Proof** We prove it by construction. Let $P$ be the matrix $S$ of eigenvectors ${x_j}$ of $A^TA$. Then, 
$$
(AS)^T(AS) = S^T (A^T A) S
$$
Recall that $A^TA$ is a symmetric matrix. Therefore, we know that if $S$ is the matrix of eigenvectors of $A^TA$ and $\Lambda$ the matrix of eigenvalues, then 
$$
\Lambda = S^T (A^T A) S
$$
Therefore, $S$ is the matrix we seek.

Shlens, A Tutorial on Principal Component Analysis

### PCA Procedure
1. Standardize each vector of observations to have mean 0.  Call the matrix of standardized observations, $A$.
2. Diagonalize the covariance matrix $A^TA = S \Lambda S^T$. $S$ is the matrix whose columns are the eigenvectors of $A^TA$ and $\Lambda$ is a diagonal matrix with the eigenvalues of $A^TA$ on the diagonal. Organize the eigenvectors of $A^TA$ from largest to smallest and rank the corresponding eigenvectors. These are your principal components.
3. Transform $A$ to the new basis provided by calculating $AS$. 

### PCA Connection to SVD
In practice, the diagonalization of $A^TA$ is usually done with Singular Value Decomposition. In the SVD form of $A$, 
$$
\mathbf{A} = \mathbf{U} \mathbf{D} \mathbf{V}^T
$$
where $\mathbf{D}$ is the diagonal matrix of eigenvalues of $A^TA$, and $V$ is the matrix of corresponding eigenvectors of $A^TA$.

## Code
### Download the data
I am using two main sources for the raw text files. The Shakespeare works come from [Folger Digital Texts][folger], and I downloaded a few other texts for comparison purposes from [Project Gutenberg][gutenberg]
* Plays: [Folger Digital Texts][folger]
 * The Folger Digital Text library provides an API that gives "stripped" versions of the play. All of the stage directions, character names, and other non-spoken material are removed. The only text left is that spoken by characters in the play. I only did some minor cleaning to remove html tags.
* Sonnets: [Folger Digital Texts][folger]
 * I did some minor cleaning to remove headings and other introductory material.
* Pride and Prejudice: [Project Gutenberg][gutenberg-PandP]
 * The Project Gutenberg text has detailed disclaimers and other legal statements at the start and end of the text. I removed this material.
* Leviathan: [Project Gutenberg][gutenberg-Leviathan]
 * This required similar cleaning as the Pride and Prejudice text.

[folger]: http://www.folgerdigitaltexts.org
[gutenberg]: http://www.gutenberg.org
[gutenberg-PandP]: http://www.gutenberg.org/files/1342/1342-0.txt
[gutenberg-Leviathan]: http://www.gutenberg.org/cache/epub/3207/pg3207.txt


In [9]:
%%capture
import download


download.shakespeare_plays("playMetadata.csv"); 
download.shakespeare_sonnets(); 
download.gutenberg_texts([('Pride and Prejudice', 'http://www.gutenberg.org/files/1342/1342-0.txt'),
                      ('Leviathan', 'http://www.gutenberg.org/cache/epub/3207/pg3207.txt')]
                     ); 

0

## Dictionary Tagging: Tag data with categories
[Docuscope] is a dictionary of words grouped by "Language Action Types" (LATs). There are mutliple versions. I used the open source one available on
[GitHub][gh Docuscope].

[Ubiqu-Ity][Ubiqu] is an academic tool for parsing texts that is specially written to work with Docuscope. It creates a vector for each text with the frequency of words used in each category. The source code is available from [GitHub][gh Ubiqu] under the BSD License. 

I execute the Docuscope tagging library through a shell script.

[Docuscope]:http://www.cmu.edu/dietrich/english/research/docuscope.html
[gh Docuscope]:https://github.com/docuscope/DocuScope-Dictionary-June-26-2012
[Ubiqu]:http://vep.cs.wisc.edu/ubiq/
[gh Ubiqu]:https://github.com/uwgraphics/Ubiqu-Ity

In [12]:
%%bash
( cd ../Ubiqu-Ity-master && python ~/msca/Ubiqu-Ity-master/Ubiqu/tagCorpus.py --ngram_per_doc --ngram_count 1 --docuscope_version default ./data ./data_tagged )

Tagging corpus data...
Starting tag_corpus...
tag_corpus finished. Total elapsed time: 429.46 seconds.


## Word Frequency
Word frequency is usually analyzed through "n-grams". An n-gram is a sequence of $n$ words. For example, for $n=2$, the line "to be or not to be" is made up of the n-grams: to be, be or, or not, not to, to be. The Ubiqu+Ity utility will also count n-grams. For my analysis, I just counted 1-grams, which means I just counted usages of individual words.  

## Load tagged data
I wrote a few Python functions to load the tagged data and clean it properly. First we use the pandas module and represent the data as a dataframe.  Each row represents a play and the columns are categories of words. The numbers represent the share of the words in that play that fall in a given category. Note that the same words can appear in multiple categories.

In [8]:
import cleanTaggedData
import pandas as pd

A, A_md = cleanTaggedData.get_tagged_texts()

A.head()

,AbstractConcepts,Acknowledge,Anger,Apology,Aside,Attack_Citation,Authoritative_Citation,Autobio,Biographical_Time,Cause,...,SubjectivePercept,SubjectiveTime,Substitution,Support,TimeDate,TimeDuration,TimeShift,Transformation,Uncertainty,Updates
0,2.015839,0.068395,0.295176,0.014399,0.140389,0.000000,0.0,0.341973,0.287977,0.187185,...,0.957523,0.284377,0.017999,0.021598,0.021598,0.867531,0.226782,0.280778,0.323974,0.561555
1,2.470064,0.091896,0.172654,0.005569,0.245057,0.000000,0.0,0.448343,0.208855,0.094681,...,1.478697,0.281259,0.005569,0.013924,0.008354,0.701754,0.323030,0.325815,0.587580,0.498468
2,1.716238,0.160772,0.144695,0.008039,0.200965,0.008039,0.0,0.546624,0.454180,0.104502,...,1.571543,0.209003,0.020096,0.008039,0.024116,0.635048,0.462219,0.377814,0.454180,0.731511
3,2.777649,0.134702,0.269404,0.018580,0.190441,0.009290,0.0,0.222955,0.246180,0.074318,...,1.272702,0.176506,0.023224,0.009290,0.004645,0.566678,0.301918,0.334433,0.469135,0.529518
4,2.682472,0.044976,0.240941,0.000000,0.131714,0.009638,0.0,0.282704,0.141352,0.109226,...,1.310717,0.212028,0.016063,0.019275,0.006425,0.462606,0.298766,0.221665,0.401568,0.815986


## Load ngram data
A second function loads a dataframe where each row is a play and each column the number of a given ngram used in that play. 

In [5]:
import cleanTaggedData
A_ngrams = cleanTaggedData.get_ngrams()
A_ngrams.head()

,limited,o’erleaps,glamis,pardon,child,needful,foul,sleek,maggot,hath,...,gipsies,theou,certaintie,ecclesia,kinde,mulct,ascarides,coupling,jewes,reneweth
1H4,NaN,NaN,NaN,0.033453,0.004182,0.004182,0.020908,NaN,NaN,0.213264,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1H6,NaN,NaN,NaN,0.009767,0.029301,0.004884,0.014651,NaN,NaN,0.253943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2H4,NaN,NaN,NaN,0.015535,0.011651,0.003884,0.015535,NaN,NaN,0.256321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2H6,NaN,NaN,NaN,0.040925,0.008185,NaN,0.040925,NaN,NaN,0.249642,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3H6,NaN,NaN,NaN,0.051553,0.034369,0.012888,0.030073,NaN,NaN,0.257765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
